In [3]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier
# from sklearn.metrics import accuracy_score, classification_report

from sklearn.impute import KNNImputer

import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
%matplotlib inline

In [4]:
df = pd.read_json('data/dataSet_Culture_21112023.json')

In [5]:
df['culture_name'].unique()

array(['Ячмень', 'Люцерна', 'Пшеница', 'Кукуруза', 'Свекла', 'Соя',
       'Томат', 'Картофель', 'Софлор', 'Лук', 'Клевер', 'Сил'],
      dtype=object)

In [6]:
df

,index_month_4,index_month_5,index_month_6,index_month_7,index_month_8,elevation,culture_name
0,-0.047,-0.298,-0.293,0.148,0.105,724,Ячмень
1,0.016,0.270,0.298,0.260,0.757,729,Люцерна
2,-0.034,-0.258,-0.017,0.131,-0.086,626,Пшеница
3,0.068,-0.186,-0.112,0.286,0.120,619,Люцерна
4,0.110,0.439,-0.300,-0.020,0.211,966,Пшеница
...,...,...,...,...,...,...,...
303,0.058,-0.223,0.284,NaN,0.187,770,Ячмень
304,0.013,0.042,0.054,NaN,0.035,763,Ячмень
305,0.028,0.006,0.064,NaN,0.037,940,Ячмень
306,-0.059,0.132,0.147,NaN,0.061,940,Ячмень


In [7]:
codes, uniques = pd.factorize(df['culture_name'])
df['culture_name'] = codes

In [8]:
culture_id_map = {name: id for id, name in enumerate(uniques)}
print(culture_id_map)

{'Ячмень': 0, 'Люцерна': 1, 'Пшеница': 2, 'Кукуруза': 3, 'Свекла': 4, 'Соя': 5, 'Томат': 6, 'Картофель': 7, 'Софлор': 8, 'Лук': 9, 'Клевер': 10, 'Сил': 11}


In [9]:
# df = df[(df['culture_name'] == 2) | (df['culture_name'] == 3) | (df['culture_name'] == 0)]

In [10]:
df['culture_name'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

True

In [37]:
imputer = KNNImputer(n_neighbors=5)
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

In [92]:
# Splitting the dfset into features (X) and target (y)
X = df.drop('culture_name', axis=1)
y = df['culture_name']

# Splitting the dfset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [93]:
df.isnull().sum()

index_month_4    0
index_month_5    0
index_month_6    0
index_month_7    0
index_month_8    0
elevation        0
culture_name     0
dtype: int64

In [94]:
df['culture_name'] = df['culture_name'].astype(int)

In [95]:
df

,index_month_4,index_month_5,index_month_6,index_month_7,index_month_8,elevation,culture_name
0,-0.047,-0.298,-0.293,0.1480,0.105,724.0,0
1,0.016,0.270,0.298,0.2600,0.757,729.0,1
2,-0.034,-0.258,-0.017,0.1310,-0.086,626.0,2
3,0.068,-0.186,-0.112,0.2860,0.120,619.0,1
4,0.110,0.439,-0.300,-0.0200,0.211,966.0,2
...,...,...,...,...,...,...,...
303,0.058,-0.223,0.284,-0.0024,0.187,770.0,0
304,0.013,0.042,0.054,0.1520,0.035,763.0,0
305,0.028,0.006,0.064,0.1730,0.037,940.0,0
306,-0.059,0.132,0.147,0.1730,0.061,940.0,0


In [96]:
rf = RandomForestClassifier(random_state=42)

In [97]:
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [98]:
rf.score(X_test, y_test)

0.43548387096774194

In [18]:
joblib.dump(rf, "my_random_forest.joblib")

['my_random_forest.joblib']

In [32]:
import warnings
from sklearn.exceptions import DataConversionWarning

# Suppress specific sklearn warnings
warnings.filterwarnings(action='ignore', category=UserWarning)
import joblib
loaded_rf = joblib.load("my_random_forest.joblib")
'''
[[NDVI_4_month, NDVI_5_month, NDVI_6_month, NDVI_7_month, NDVI_8_month, elavation_countour]]
'''
sample_data = [[0.14, 0.658, 0.331, -0.129, 0.191, 664.0]]

# Make a prediction
prediction = loaded_rf.predict(sample_data)

# Dictionary mapping encoded values to names
name_mapping = {0: 'Ячмень', 1: 'Люцерна', 2: 'Пшеница', 3: 'Кукуруза', 4: 'Свекла', 5: 'Соя', 6: 'Томат', 7: 'Картофель', 8: 'Софлор', 9: 'Лук', 10: 'Клевер', 11: 'Сил'}

# Convert encoded prediction to name
predicted_name = name_mapping[int(prediction[0])]

print(predicted_name)

Ячмень
